# Attack on networks!

In this assignment, we will break a network into pieces. Many empirical networks are hard to break if we randomly attack it. However, by strategically attacking a specific part of the network, we can break it very easily. This leads to the idea of "importance", i.e., a node is "important" for the network if removing the node breaks the network. 

#### Network data

We will use the worldwide airport network based on data taken from Openflight.org.

In [ ]:
import pandas as pd
import igraph
import numpy as np

node_table = pd.read_csv(
    "https://raw.githubusercontent.com/skojaku/adv-net-sci-course/main/data/airport_network_v2/node_table.csv"
)
edge_table = pd.read_csv(
    "https://raw.githubusercontent.com/skojaku/adv-net-sci-course/main/data/airport_network_v2/edge_table.csv"
)
src, trg = tuple(edge_table[["src", "trg"]].values.T)
edge_list = tuple(zip(src, trg))

# node_id and name dictionary
n_nodes = node_table.shape[0]
id2name = np.array([""] * n_nodes, dtype="<U64")
id2name[node_table["node_id"]] = node_table["Name"].values

g = igraph.Graph(
    edge_list,
    vertex_attrs=dict(Name=id2name, node_id=node_table["node_id"].values),
)

# You can retrieve the airport names by
print(g.vs[0]["Name"], ",", g.vs[1]["Name"], ", ...")

# Centrality 

Centrality is a central metric in network analysis used to determine the importance of a node within a network based on its structural position. It can also be applied to individual edges, measuring the significance of a link within the network. Numerous centrality metrics have been developed, and in this assignment, we will explore some commonly used centrality metrics.

## Degree centrality

The simplest metric of centrality is degree centrality. The centrality of a node is the degree of the node. And with igraph, you can get the degree of nodes by

In [ ]:
degree_centrality = g.degree()

## Path-based centrality

#### Closeness centrality

One way to measure the importance of nodes is to measure how central they are in the network. A node at the center of the network should be able to reach any other nodes in *short* distances. One way to quantify this is to take the sum of the distances from individual nodes to all other nodes in the network, with a simple inversion to make "central" nodes having a high centrality, i.e.,
$$
c_i:= \frac{n-1}{\sum_{j, i\neq j} d(i,j)},
$$
where $d(i,j)$ is the shortest path length between nodes $i$ and $j$, and $n$ is the number of nodes in the network. This metric is called *closeness centrality*. The numerator serves as a scaling constant to make the metric vary in the range $(0,1]$. With `igraph`, it can be computed by


In [ ]:
closeness_centrality = g.closeness()

### Betweenness centrality

Betweenness centrality is another widespread centrality based on the shortest path lengths. It is defined by
$$
b_i:= \sum_{s \neq t\neq i} \frac{\sigma_{s,i,t}}{\sigma_{s,t}}
$$
where $\sigma_{s,t}$ is the number of shortest paths between node $s$ and node $v$, and $\sigma_{s,i,t}$ is the shortest paths between $s$ and $t$ that goes through node $i$. A node with a high betweenness centrality means that the node is a dominant intermediary of flows (that go through the shortest path) between nodes on the network. Or, you can think of it as a bottleneck of flows. With igraph, you can compute the betweenness centrality by

In [ ]:
betweenness_centrality = g.closeness()

The idea of betweenness centrality can be extended to edges. Instead of counting the number of shortest paths going through nodes, we can count them for each edge and define the betweenness centrality in the same way. You can compute the edge betweenness centrality with igraph by

In [ ]:
edge_betweeness_centrality = g.edge_betweenness()  # compute the betweenness centrality

# You can retrieve the edge lists pertaining to the edge betweenness centrality by
edge_list = g.get_edgelist()

# For example, the betweenness centrality edge_betweeness_centrality[10] corresponds to an edge edge_list[10]

# Robustness

Another approach to determining the importance of nodes is by evaluating the impact their removal would have on the network. For instance, an individual can be deemed important within a social network if their departure leads to the network becoming fragmented into disconnected segments. Similarly, certain Internet routers are considered critical to the functioning of the Internet, as their removal would result in the malfunctioning of systems.

We can consider two types of network damage: random failure and target attack. In a random failure scenario, nodes are selected at random for removal. However, in a targeted attack, nodes are strategically chosen for removal based on specific criteria. For instance, nodes may be removed in the descending order of their degree centrality.


## Random failure

Let's break the network 😈! Before damaging the network, we should damage a copy of the network while keeping the original network as it is. This way, we can come back to the original network and compare it against the damaged one.

In [ ]:
g_damaged = g.copy()

# And also it is good to keep the original node_id as the node attributes.
# In igraph, the node attribuets are stored in .vs attributes. It's like a dictionary.
# You can get the original node_ids after damaging the network by g_damaged.vs()["node_id"].
g_damaged.vs["node_id"] = np.arange(g.vcount(), dtype=int)

You can remove a node from the graph by using .delete_vertices API. 

In [ ]:
nodes_to_be_removed = [1, 3, 5]
g_damaged.delete_vertices(nodes_to_be_removed)